In [1]:
!pip install mediapipe opencv-python

     ---------------------------------------- 50.1/50.1 MB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 38.2/38.2 MB 3.5 MB/s eta 0:00:00
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
     -------------------------------------- 904.0/904.0 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 44.9/44.9 MB 3.6 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing=mp.solutions.drawing_utils #visualising poses
mp_pose=mp.solutions.pose #importing pose estimation model

In [3]:
def angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    rad=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    degree=np.abs(rad*180/np.pi)
    
    angle=min(degree,360-degree)
    return int(angle)

In [14]:
cap = cv2.VideoCapture(0)

#Training Monitoring
counter=0
stage=None
bool_torso=None
bool_foot=None

# Initialize Mediapipe Pose model
with mp.solutions.pose.Pose() as pose:
    while cap.isOpened():
        # Read frame from Webcam
        success, image = cap.read()
        if not success:
            break

        # Convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process image with Mediapipe Pose model
        results = pose.process(image)
        
        #Recolor to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Extract Landmarks
        try:
            landmarks=results.pose_landmarks.landmark
            
            #Compute angle
            left_hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee=[landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle=[landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            knee_angle=angle(left_hip,left_knee,left_ankle)
            
            left_heel=[landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            left_foot_index=[landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            foot_angle=angle(left_knee,left_heel,left_foot_index)
            
            left_hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            torso_angle=angle(left_knee,left_hip,left_shoulder)
            
            #Output Angle
            cv2.putText(image, str(knee_angle),tuple(np.multiply(left_knee,[1280,720]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2,cv2.LINE_AA)
            
            #Counter Logic
            if knee_angle>170:
                stage="up"
            if knee_angle<100 and stage=="up":
                stage="down"
                counter+=1
                
            #Pose Correction Logic
            if foot_angle<60:
                bool_foot=1
            else:
                bool_foot=0
                
            if torso_angle<60:
                bool_torso=1
            else:
                bool_torso=0
            
        except:
            pass
        
        #Display Counter
        cv2.rectangle(image,(0,0),(150,75),(0,0,0),-1)
        cv2.rectangle(image,(0,450),(640,480),(0,0,0),-1)
        cv2.putText(image,"REPS",(10,10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,str(counter),(15,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
        if bool_torso==1:
            cv2.putText(image,"Upper body hunched forward",(15,465),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            
        if bool_foot==1:
            cv2.putText(image,"Knees leaning forward",(320,465),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)


        # Draw pose landmarks
        if results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                                     mp_drawing.DrawingSpec(color=(225,105,65), thickness=2,circle_radius=2),
                                                     mp_drawing.DrawingSpec(color=(180,229,255), thickness=2,circle_radius=2))

        # Display webcam feed
        cv2.imshow('Feed',image)
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [13]:
import pickle
op=[counter, bool_torso, bool_foot]
op

[2, 0, 0]